In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cv-3-bitcoin (use `wandb login --relogin` to force relogin)


True

In [2]:
import os
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.7.0 True
11.0
GCC 7.3


In [3]:
# 모듈 import
import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)

In [4]:
seed = 42

set_random_seed(seed, deterministic=False)
os.environ['PYTHONHASHSEED'] = str(seed)

In [5]:
from mmcv import Config

# config file 들고오기
baseline_cfg_path = './configs/vfnet/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco.py'
cfg = Config.fromfile(baseline_cfg_path)

In [6]:
model_name = 'vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco'
exp = 'basic'

# Folder to store model logs and weight files
job_folder = f'./working/exp[{exp}]_{model_name}'
cfg.work_dir = job_folder

# Change the wnd username and project name below
wnb_username = 'sunhyuk'
wnb_project_name = 'cv-3-bitcoin'

if not os.path.exists(job_folder):
    os.makedirs(job_folder)

print("Job folder:", job_folder)

Job folder: ./working/exp[basic]_vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco


In [7]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
root='../../dataset/'

albu_train_transforms=[
    dict(
        type='HorizontalFlip',
        p=0.5,
    ),
    dict(
        type='VerticalFlip',
        p=0.5,
    ),
    dict(
        type='RandomBrightnessContrast',
        brightness_limit=[0.1, 0.3],
        contrast_limit=[0.1, 0.3],
        p=0.3),
    dict(
        type='HueSaturationValue',
        p=0.3,
    ),
    dict(
        dict(type="IAAAdditiveGaussianNoise"),
        p=0.3,
    ),
    dict(
        type='Sharpen',
        p=0.3,
    ),
    dict(
        type='RandomSizedBBoxSafeCrop',
        width=768,
        height=768,
        p=0.1
    )
]
# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'cv_train_1.json' # train json 정보
cfg.data.train.pipeline[2] = dict(
    type="Resize",
    img_scale=[(1024, 1024)],
    multiscale_mode="range",
    ratio_range=(0.5, 2.0),
    keep_ratio=True,
)
cfg.data.train.pipeline.insert(
    3, dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type="BboxParams",
            format="pascal_voc",
            label_fields=["gt_labels"],
            min_visibility=0.0,
            filter_lost_elements=True,
        ),
        keymap={"img": "image", "gt_bboxes": "bboxes"},
        update_pad_shape=False,
    )
)
                               


cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'cv_val_1.json' # test json 정보
cfg.data.val.pipeline[1]['img_scale'] = (1024,1024) # Resize
del cfg.data.val.pipeline[1]['transforms'][1]

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (1024,1024) # Resize
del cfg.data.test.pipeline[1]['transforms'][1]


cfg.data.samples_per_gpu = 1
# cfg.data.workers_per_gpu = 0

cfg.seed = seed #~~~
cfg.gpu_ids = [0]
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

# cfg.model.roi_head.bbox_head.num_classes = 10 ##클래수 수
cfg.model.bbox_head.num_classes = 10

cfg.evaluation = dict(
    interval=1,
    save_best='auto',
    metric='bbox',
    classwise=True,
)

#
cfg.optimizer = dict(type='SGD', lr=0.001, momentum=0.9, weight_decay=0.0001)
# cfg.optimizer = dict(type='Adam', lr=3e-4)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
# cfg.lr_config = dict(
#     policy='step',
#     warmup='linear',
#     warmup_iters=50,
#     warmup_ratio=0.001,
#     step=[8, 11]
# )


# cfg.runner.max_epochs = 12

wandb_cfg = dict(config={"epochs": cfg.runner.max_epochs, "batch_size": cfg.data.samples_per_gpu, "optimizer":cfg.optimizer, "model": cfg.model, "img_norm_cfg" : cfg.img_norm_cfg, "train_pipeline" : cfg.train_pipeline, "test_pipeline" : cfg.test_pipeline, "data" : cfg.data, "lr_config" : cfg.lr_config})

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         interval=10,
         init_kwargs=dict(config=wandb_cfg, project=wnb_username,entity=wnb_project_name, name=f'[{exp}]_{model_name}')
        )
]
cfg.log_config.interval = 10
# cfg.checkpoint_config = 20
print(cfg.pretty_text)


dataset_type = 'CocoDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(1333, 480), (1333, 960)],
        multiscale_mode='range',
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
     

In [8]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

# dataset 확인
datasets[0]

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


/opt/conda/envs/mymmdet/lib/python3.7/site-packages/albumentations/imgaug/transforms.py:252: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)



CocoDataset Train dataset with number of images 3677, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 2993  | 1 [Paper]     | 4685  | 2 [Paper pack]  | 681   | 3 [Metal]   | 711   | 4 [Glass]    | 733   |
| 5 [Plastic]       | 2211  | 6 [Styrofoam] | 970   | 7 [Plastic bag] | 3808  | 8 [Battery] | 136   | 9 [Clothing] | 378   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [9]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-04-05 20:12:03,323 - mmcv - INFO - initialize ResNeXt with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://resnext101_64x4d'}
2022-04-05 20:12:03,324 - mmcv - INFO - load model from: open-mmlab://resnext101_64x4d
2022-04-05 20:12:03,325 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://resnext101_64x4d
2022-04-05 20:12:03,595 - mmcv - WARNING - The model and loaded state dict do not match exactly

missing keys in source state_dict: layer2.0.conv2.conv_offset.weight, layer2.0.conv2.conv_offset.bias, layer2.1.conv2.conv_offset.weight, layer2.1.conv2.conv_offset.bias, layer2.2.conv2.conv_offset.weight, layer2.2.conv2.conv_offset.bias, layer2.3.conv2.conv_offset.weight, layer2.3.conv2.conv_offset.bias, layer3.0.conv2.conv_offset.weight, layer3.0.conv2.conv_offset.bias, layer3.1.conv2.conv_offset.weight, layer3.1.conv2.conv_offset.bias, layer3.2.conv2.conv_offset.weight, layer3.2.conv2.conv_offset.bias, layer3.3.conv2.conv_offset.weight, layer3.3.conv2.conv_

In [ ]:
# 모델 학습
train_detector(model, datasets[0], cfg=cfg, distributed=False, validate=True)

2022-04-05 20:12:07,034 - mmdet - INFO - Start running, host: root@642ef2a3bc92, work_dir: /opt/ml/detection/baseline/mmdetection/working/exp[basic]_vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco
2022-04-05 20:12:07,035 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


/opt/conda/envs/mymmdet/lib/python3.7/site-packages/mmcv/runner/hooks/evaluation.py:222: UserWarning: runner.meta is None. Creating an empty one.
  warnings.warn('runner.meta is None. Creating an empty one.')


2022-04-05 20:12:25,437 - mmdet - INFO - Epoch [1][10/3677]	lr: 1.162e-04, eta: 1 day, 12:35:55, time: 1.493, data_time: 0.224, memory: 12841, loss_cls: 0.8300, loss_bbox: 0.9221, loss_bbox_rf: 1.2548, loss: 3.0069, grad_norm: 58.1772
2022-04-05 20:12:39,971 - mmdet - INFO - Epoch [1][20/3677]	lr: 1.342e-04, eta: 1 day, 12:06:29, time: 1.453, data_time: 0.006, memory: 17553, loss_cls: 0.5822, loss_bbox: 0.8014, loss_bbox_rf: 1.4382, loss: 2.8218, grad_norm: 50.9459
2022-04-05 20:12:53,958 - mmdet - INFO - Epoch [1][30/3677]	lr: 1.522e-04, eta: 1 day, 11:29:36, time: 1.399, data_time: 0.005, memory: 17553, loss_cls: 0.8164, loss_bbox: 0.7104, loss_bbox_rf: 1.2554, loss: 2.7822, grad_norm: 48.3172
2022-04-05 20:13:07,005 - mmdet - INFO - Epoch [1][40/3677]	lr: 1.702e-04, eta: 1 day, 10:36:32, time: 1.305, data_time: 0.005, memory: 17553, loss_cls: 0.8589, loss_bbox: 0.7717, loss_bbox_rf: 1.1882, loss: 2.8188, grad_norm: 33.1396
2022-04-05 20:13:21,571 - mmdet - INFO - Epoch [1][50/3677]	

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1206/1206, 4.0 task/s, elapsed: 302s, ETA:     0s

...

2022-04-06 02:26:42,821 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.32s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.53s).
Accumulating evaluation results...


2022-04-06 02:26:53,726 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.038
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.059
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.040
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.012
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.023
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.047
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.377
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.377
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.377
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.012
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.143
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=3.48s).


2022-04-06 02:26:54,012 - mmdet - INFO - The previous best checkpoint /opt/ml/detection/baseline/mmdetection/working/exp[basic]_vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco/best_bbox_mAP_epoch_3.pth was removed
2022-04-06 02:26:56,235 - mmdet - INFO - Now best checkpoint is saved as best_bbox_mAP_epoch_4.pth.
2022-04-06 02:26:56,236 - mmdet - INFO - Best bbox_mAP is 0.0380 at 4 epoch.
2022-04-06 02:26:56,242 - mmdet - INFO - Epoch(val) [4][1206]	bbox_mAP: 0.0380, bbox_mAP_50: 0.0590, bbox_mAP_75: 0.0400, bbox_mAP_s: 0.0120, bbox_mAP_m: 0.0230, bbox_mAP_l: 0.0470, bbox_mAP_copypaste: 0.038 0.059 0.040 0.012 0.023 0.047
2022-04-06 02:27:10,575 - mmdet - INFO - Epoch [5][10/3677]	lr: 1.000e-03, eta: 1 day, 5:22:02, time: 1.432, data_time: 0.228, memory: 17553, loss_cls: 1.3400, loss_bbox: 0.3671, loss_bbox_rf: 0.4629, loss: 2.1700, grad_norm: 7.2578
2022-04-06 02:27:24,977 - mmdet - INFO - Epoch [5][20/3677]	lr: 1.000e-03, eta: 1 day, 5:21:48, time: 1.440, data_time: 0.006, memory: 1